## Debugging 함수 정의

In [15]:
import os
import re

def is_weird(values):
    # 값이 1~5 범위 내의 숫자인지 확인
    out_of_range = False
    for val in values:
        if not val.isdigit() or not (1 <= int(val) <= 5):
            out_of_range = True
            break
    # 중복된 값이 있는지 확인
    has_duplicates = len(values) != len(set(values))
    return has_duplicates, out_of_range

def process_file(input_file, output_file, user_question_counts):
    users_with_issues = []
    user_numbers = []
    output_data = []
    user_issue_reasons = {}  # 사용자별 오류 이유 저장

    with open(input_file, 'r', encoding='utf-8') as infile:

        current_user = None
        current_user_id = None
        questions_seen = set()
        user_has_issue = False
        user_data = []
        issue_reasons = set()

        for line in infile:
            line = line.strip()
            if line.startswith('[U') and line.endswith(']'):
                # 이전 사용자 처리
                if current_user is not None:
                    # 예상 질문 수와 실제 질문 수 비교
                    expected_question_count = user_question_counts.get(current_user_id)
                    actual_question_count = len(questions_seen)
                    if expected_question_count is not None and actual_question_count != expected_question_count:
                        user_has_issue = True
                        issue_reasons.add('Question 부족')
                    if user_has_issue:
                        users_with_issues.append(current_user)
                        user_numbers.append(current_user_id[1:])  # 'U'를 제외한 번호만 추가
                        # 오류 이유 저장
                        user_issue_reasons[current_user] = ', '.join(issue_reasons)
                        output_data.extend(user_data)
                    # 초기화
                    user_has_issue = False
                    issue_reasons = set()
                    questions_seen = set()
                    user_data = []
                # 새로운 사용자 초기화
                current_user = line
                user_data.append(line)
                user_num_match = re.match(r'\[U(\d+)\]', current_user)
                if user_num_match:
                    user_number = user_num_match.group(1)
                    current_user_id = 'U' + user_number
                else:
                    current_user_id = None
            elif line.startswith('Question'):
                # 질문 번호와 값 추출
                question_match = re.match(r'Question\s+(\d+)\s*:\s*(.+)', line)
                if question_match:
                    question_num = question_match.group(1)
                    values = [v.strip() for v in question_match.group(2).split(',')]
                    # 질문 번호 중복 확인
                    if question_num in questions_seen:
                        user_has_issue = True
                        issue_reasons.add('중복된 질문 번호')
                        user_data.append(line)
                    else:
                        questions_seen.add(question_num)
                        # 값 검사
                        if len(values) != 5:
                            user_has_issue = True
                            issue_reasons.add('잘못된 형식')
                        has_duplicates, out_of_range = is_weird(values)
                        if has_duplicates:
                            user_has_issue = True
                            issue_reasons.add('중복된 값')
                        if out_of_range:
                            user_has_issue = True
                            issue_reasons.add('범위 초과')
                        user_data.append(line)
                else:
                    # 형식이 맞지 않는 경우
                    user_has_issue = True
                    issue_reasons.add('잘못된 형식')
                    user_data.append(line)
            elif line == '':
                continue
            else:
                # 예상치 못한 형식의 데이터 처리
                user_has_issue = True
                issue_reasons.add('잘못된 형식')
                user_data.append(line)

        # 마지막 사용자 처리
        if current_user is not None:
            expected_question_count = user_question_counts.get(current_user_id)
            actual_question_count = len(questions_seen)
            if expected_question_count is not None and actual_question_count != expected_question_count:
                user_has_issue = True
                issue_reasons.add('Question 부족')
            if user_has_issue:
                users_with_issues.append(current_user)
                user_numbers.append(current_user_id[1:])  # 'U'를 제외한 번호만 추가
                user_issue_reasons[current_user] = ', '.join(issue_reasons)
                output_data.extend(user_data)

    # 출력 파일에 작성
    with open(output_file, 'w', encoding='utf-8') as outfile:
        if users_with_issues:
            # 오류 사용자 목록과 총 사용자 수 작성
            outfile.write('오류 USER: ' + ', '.join(f"{u} : {user_issue_reasons[u]}" for u in users_with_issues) + '\n')
            outfile.write('USER list: [' + ', '.join(user_numbers) + ']\n')
            outfile.write('총 오류 USER: ' + str(len(users_with_issues)) + '\n\n')
            # 각 사용자별 데이터 출력
            for user in users_with_issues:
                outfile.write(f"{user} : {user_issue_reasons[user]}\n")
                # 해당 사용자 데이터 추출
                user_lines = []
                user_started = False
                for line in output_data:
                    if line == user:
                        user_started = True
                        continue
                    if line.startswith('[U') and user_started:
                        break
                    if user_started:
                        outfile.write(line + '\n')
                outfile.write('\n')
        else:
            outfile.write('오류가 발견되지 않았습니다.\n')

def read_meta_file(meta_file):
    user_question_counts = {}
    with open(meta_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            # 정규식을 사용하여 User ID와 Question 수 추출
            match = re.match(r'User ID:\s*(U\d+).*Question 수:\s*(\d+)', line)
            if match:
                user_id = match.group(1)
                question_count = int(match.group(2))
                user_question_counts[user_id] = question_count
            else:
                # print(f"메타 파일에서 라인을 파싱할 수 없습니다: {line}")
                pass
    return user_question_counts

def process_debugging(input, purpose):
    
    input_file = f'result/gpt_result/{input}'
    meta_file = f'user_prompts/{purpose}/metadata/output_metadata.txt'
    output_file = f'result/error_detect/{input.replace(".txt", "")}.txt'
    
    # 메타 파일 읽기
    user_question_counts = read_meta_file(meta_file)
    # 출력 디렉토리가 없으면 생성
    output_dir = os.path.dirname(output_file)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    process_file(input_file, output_file, user_question_counts)
    print(f"{input_file} 파일을 처리 완료하였습니다.\n저장 파일 : {output_file}\n")


## 실행

In [16]:
process_debugging('positive_finetuned_2.txt', 'only_positive')
process_debugging('2_negative_finetuned.txt', 'with_negative')

result/gpt_result/positive_finetuned_2.txt 파일을 처리 완료하였습니다.
저장 파일 : result/error_detect/positive_finetuned_2.txt

result/gpt_result/2_negative_finetuned.txt 파일을 처리 완료하였습니다.
저장 파일 : result/error_detect/2_negative_finetuned.txt

